In [1]:
import pandas as pd
import numpy as np
import category_encoders as ce
# from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# from imblearn.over_sampling import SMOTE
from sklearn.linear_model import SGDRegressor

RANDOM_STATE = 42


In [2]:
df = pd.read_csv('DoctorFeePrediction.csv')

In [3]:
df.head()

,Doctor Name,City,Specialization,Doctor Qualification,Experience(Years),Total_Reviews,Patient Satisfaction Rate(%age),Avg Time to Patients(mins),Wait Time(mins),Hospital Address,Doctors Link,Fee(PKR)
0,Dr. Umair Hafeez,GUJRANWALA,Ent Specialist,"MBBS , FCPS ( Otorhinolaryngologic ENT )",6.0,11,100,19,6,"Central Hospital, Jinnah Colony, Gujranwala",https://www.marham.pk/doctors/gujranwala/ent-s...,2000
1,Dr. Haris Shakeel,RAJAN-PUR,General Physician,MBBS,1.0,0,94,14,11,No Address Available,No Link Available,500
2,Dr. Iqra Rehman,MIRPUR-KHAS,General Physician,"MBBS, RMP, CFP (USA), Certified in Covid 19 +",6.0,9,100,10,0,"Rehman Clinic, tandoadam naka, Mirpur Khas",https://www.marham.pk/doctors/mirpur-khas/gene...,1000
3,Dr. Erum Memon,HYDERABAD,Gynecologist,"MBBS, FCPS (Gynae &amp; Obstetrics)",11.0,71,96,18,10,"Mehmood Hospital, Qasimabad, Hyderabad",https://www.marham.pk/doctors/hyderabad/gyneco...,800
4,Dr. Aisha Ahmad,LAHORE,Dermatologist,"MBBS, FCPS (Dermatology), Certified (Aesthetic...",12.0,199,100,16,2,"Skinnovation, Johar Town, Lahore",https://www.marham.pk/doctors/lahore/dermatolo...,1500


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2386 entries, 0 to 2385
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Doctor Name                      2386 non-null   object 
 1   City                             2386 non-null   object 
 2   Specialization                   2386 non-null   object 
 3   Doctor Qualification             2386 non-null   object 
 4   Experience(Years)                2386 non-null   float64
 5   Total_Reviews                    2386 non-null   int64  
 6   Patient Satisfaction Rate(%age)  2386 non-null   int64  
 7   Avg Time to Patients(mins)       2386 non-null   int64  
 8   Wait Time(mins)                  2386 non-null   int64  
 9   Hospital Address                 2386 non-null   object 
 10  Doctors Link                     2386 non-null   object 
 11  Fee(PKR)                         2386 non-null   int64  
dtypes: float64(1), int64

In [5]:
df.describe()

,Experience(Years),Total_Reviews,Patient Satisfaction Rate(%age),Avg Time to Patients(mins),Wait Time(mins),Fee(PKR)
count,2386.000000,2386.000000,2386.000000,2386.000000,2386.000000,2386.000000
mean,11.844719,92.473177,96.657586,14.092205,11.264459,1201.016764
std,8.784449,282.162526,4.962300,2.722198,5.636885,831.760021
min,1.000000,0.000000,33.000000,3.000000,0.000000,0.000000
25%,6.000000,0.000000,94.000000,14.000000,10.000000,600.000000
50%,10.000000,8.000000,98.000000,14.000000,11.000000,1000.000000
75%,14.000000,54.000000,100.000000,15.000000,11.000000,1500.000000
max,53.000000,5147.000000,100.000000,50.000000,82.000000,10000.000000


In [6]:
df['Fee(PKR)'].unique()

array([ 2000,   500,  1000,   800,  1500,  2500,   700,  1200,   400,
         300,  1400,  5000,   600,   250,  4000,  1799,   200,   180,
        3500,  3000,   350,  2200,   900,   750,  2800,     0,  1800,
        1300,   150,  2600,     5,   999,  2700,   450,   100,    50,
        1700,  1600,   850, 10000,    10,  1100,    25,    20,  1499,
        2100,  2400,  7500], dtype=int64)

In [7]:
df['Fee(PKR)'].value_counts()

Fee(PKR)
1000     617
500      350
1500     327
2000     262
800      108
700      102
300       79
3000      78
2500      65
1200      65
600       60
400       42
200       33
2200      24
0         22
5         17
4000      16
750       15
3500      13
5000      13
1400      11
250       11
1800       7
900        6
1300       6
100        5
1600       4
50         3
2700       2
10         2
10000      2
1799       2
350        2
1499       1
20         1
25         1
2400       1
1100       1
2100       1
2800       1
850        1
1700       1
450        1
999        1
2600       1
150        1
180        1
7500       1
Name: count, dtype: int64

In [8]:
print(df.shape)
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True)
# Costing Features
df=df.drop(['Doctor Name'],axis=1)
df=df.drop(['index'],axis=1)
print(df.shape)


(2386, 12)
(2373, 11)


In [9]:
first_split = df['Hospital Address'].str.split(',', expand=True)
second_split = first_split[0].str.split(';', expand=True)
new_column_zero = second_split.iloc[:, 0]
df_concatenated = pd.concat([df, new_column_zero], axis=1)
df = df_concatenated.rename(columns={0: 'Hospital Name'})
df.drop('Hospital Address', axis=1, inplace=True)

In [10]:
df.head()

,City,Specialization,Doctor Qualification,Experience(Years),Total_Reviews,Patient Satisfaction Rate(%age),Avg Time to Patients(mins),Wait Time(mins),Doctors Link,Fee(PKR),Hospital Name
0,GUJRANWALA,Ent Specialist,"MBBS , FCPS ( Otorhinolaryngologic ENT )",6.0,11,100,19,6,https://www.marham.pk/doctors/gujranwala/ent-s...,2000,Central Hospital
1,RAJAN-PUR,General Physician,MBBS,1.0,0,94,14,11,No Link Available,500,No Address Available
2,MIRPUR-KHAS,General Physician,"MBBS, RMP, CFP (USA), Certified in Covid 19 +",6.0,9,100,10,0,https://www.marham.pk/doctors/mirpur-khas/gene...,1000,Rehman Clinic
3,HYDERABAD,Gynecologist,"MBBS, FCPS (Gynae &amp; Obstetrics)",11.0,71,96,18,10,https://www.marham.pk/doctors/hyderabad/gyneco...,800,Mehmood Hospital
4,LAHORE,Dermatologist,"MBBS, FCPS (Dermatology), Certified (Aesthetic...",12.0,199,100,16,2,https://www.marham.pk/doctors/lahore/dermatolo...,1500,Skinnovation


In [11]:
def encode_link(link):
    if link != 'No Link Available':
        return 1
    else:
        return 0

# Apply the function to encode the 'Doctors Link' column
df['Doctors Link'] = df['Doctors Link'].apply(lambda x: encode_link(x))

# Now 'Doctors Link' column contains 1 if a link is available, 0 otherwise


In [12]:
df.head()

,City,Specialization,Doctor Qualification,Experience(Years),Total_Reviews,Patient Satisfaction Rate(%age),Avg Time to Patients(mins),Wait Time(mins),Doctors Link,Fee(PKR),Hospital Name
0,GUJRANWALA,Ent Specialist,"MBBS , FCPS ( Otorhinolaryngologic ENT )",6.0,11,100,19,6,1,2000,Central Hospital
1,RAJAN-PUR,General Physician,MBBS,1.0,0,94,14,11,0,500,No Address Available
2,MIRPUR-KHAS,General Physician,"MBBS, RMP, CFP (USA), Certified in Covid 19 +",6.0,9,100,10,0,1,1000,Rehman Clinic
3,HYDERABAD,Gynecologist,"MBBS, FCPS (Gynae &amp; Obstetrics)",11.0,71,96,18,10,1,800,Mehmood Hospital
4,LAHORE,Dermatologist,"MBBS, FCPS (Dermatology), Certified (Aesthetic...",12.0,199,100,16,2,1,1500,Skinnovation


In [13]:
df.shape

(2373, 11)

In [14]:
popular_degrees = ['MBBS', 'FCPS', 'MCPS', 'MS', 'MD', 'FRCS']
qualifications_df = pd.DataFrame({'MBBS':[], 'FCPS':[], 'MCPS':[], 'MS':  [], 'MD':  [], 'FRCS':[]})
qualifications_df.head()

,MBBS,FCPS,MCPS,MS,MD,FRCS


NOTE: Doctors with non of these qualifications should have his columns set to 0 instead of being dropped

In [15]:
for  index in range(df.shape[0]):
    current_qualifications = df.iloc[index]['Doctor Qualification']
    row = []
    for qualification in popular_degrees:
        if current_qualifications.find(qualification) != -1:
            row.append(1)
        else:
            row.append(0)
    qualifications_df.loc[len(qualifications_df)] = row 
    
qualifications_df.head()

,MBBS,FCPS,MCPS,MS,MD,FRCS
0,1,1,0,0,0,0
1,1,0,0,0,0,0
2,1,0,0,0,0,0
3,1,1,0,0,0,0
4,1,1,0,0,0,0


In [16]:
qualifications_df.shape

(2373, 6)

In [17]:
df[popular_degrees] = qualifications_df.copy()

In [18]:
df.head()

,City,Specialization,Doctor Qualification,Experience(Years),Total_Reviews,Patient Satisfaction Rate(%age),Avg Time to Patients(mins),Wait Time(mins),Doctors Link,Fee(PKR),Hospital Name,MBBS,FCPS,MCPS,MS,MD,FRCS
0,GUJRANWALA,Ent Specialist,"MBBS , FCPS ( Otorhinolaryngologic ENT )",6.0,11,100,19,6,1,2000,Central Hospital,1,1,0,0,0,0
1,RAJAN-PUR,General Physician,MBBS,1.0,0,94,14,11,0,500,No Address Available,1,0,0,0,0,0
2,MIRPUR-KHAS,General Physician,"MBBS, RMP, CFP (USA), Certified in Covid 19 +",6.0,9,100,10,0,1,1000,Rehman Clinic,1,0,0,0,0,0
3,HYDERABAD,Gynecologist,"MBBS, FCPS (Gynae &amp; Obstetrics)",11.0,71,96,18,10,1,800,Mehmood Hospital,1,1,0,0,0,0
4,LAHORE,Dermatologist,"MBBS, FCPS (Dermatology), Certified (Aesthetic...",12.0,199,100,16,2,1,1500,Skinnovation,1,1,0,0,0,0


In [19]:
df.isna().count()

City                               2373
Specialization                     2373
Doctor Qualification               2373
Experience(Years)                  2373
Total_Reviews                      2373
Patient Satisfaction Rate(%age)    2373
Avg Time to Patients(mins)         2373
Wait Time(mins)                    2373
Doctors Link                       2373
Fee(PKR)                           2373
Hospital Name                      2373
MBBS                               2373
FCPS                               2373
MCPS                               2373
MS                                 2373
MD                                 2373
FRCS                               2373
dtype: int64

In [20]:
df.rename(columns={'Experience(Years)':'EXP(YRs)',
                   'Total_Reviews' : '#Reviews',
                   f'Patient Satisfaction Rate(%age)': 'Satisfaction Rate'}, inplace=True)


In [21]:
df.shape

(2373, 17)

In [22]:
# X_ = df.drop('Fee(PKR)', axis=1)
# y_ = df['Fee(PKR)']
train, test = train_test_split(df, train_size=0.8, random_state=RANDOM_STATE)

train.reset_index(inplace=True)
test.reset_index(inplace=True)
train.drop(['index'], axis=1, inplace=True)
test.drop(['index'], axis=1, inplace=True)

In [23]:
train.head()

,City,Specialization,Doctor Qualification,EXP(YRs),#Reviews,Satisfaction Rate,Avg Time to Patients(mins),Wait Time(mins),Doctors Link,Fee(PKR),Hospital Name,MBBS,FCPS,MCPS,MS,MD,FRCS
0,SAHIWAL,Orthopedic Surgeon,"MBBS, FCPS (Orthopedic Surgery)",13.0,8,100,11,14,1,1500,Saeed surgical complex,1,1,0,0,0,0
1,ISLAMABAD,Nephrologist,"MBBS, FCPS (Nephrology)",10.0,32,94,13,7,1,1200,Advanced International Hospital,1,1,0,0,0,0
2,GUJRAT,"General Practitioner, Gynecologist",MBBS,12.0,0,94,14,11,0,500,No Address Available,1,0,0,0,0,0
3,SAHIWAL,General Physician,MBBS,2.0,12,100,14,11,1,2000,No Address Available,1,0,0,0,0,0
4,LAHORE,Pediatrician,"MBBS , DCH , MRCPCH (UK) , FRCPCH (UK)",20.0,122,97,15,14,1,1500,Hameedah Memorial Hospital,1,0,0,0,0,0


In [24]:
type(test)

pandas.core.frame.DataFrame

In [25]:
# OE = OrdinalEncoder(categories=[['Cheap', 'Medium-Priced', 'Expensive']])
# train['Fee(PKR)'] = OE.fit_transform(train[['Fee(PKR)']])
# test['Fee(PKR)'] = OE.transform(test[['Fee(PKR)']])
mean_encoder_c, mean_encoder_s, mean_encoder_h = ce.TargetEncoder(),ce.TargetEncoder(), ce.TargetEncoder()

mean_encoder_c = ce.TargetEncoder()
mean_encoder_c.fit(train[['City']], train['Fee(PKR)'])
train['City'] = mean_encoder_c.transform(train['City'])
test['City'] = mean_encoder_c.transform(test['City'])
mean_encoder_s.fit(train['Specialization'], train['Fee(PKR)'])
train['Specialization'] = mean_encoder_s.transform(train['Specialization'])
test['Specialization'] = mean_encoder_s.transform(test['Specialization'])


In [26]:
train.head()

,City,Specialization,Doctor Qualification,EXP(YRs),#Reviews,Satisfaction Rate,Avg Time to Patients(mins),Wait Time(mins),Doctors Link,Fee(PKR),Hospital Name,MBBS,FCPS,MCPS,MS,MD,FRCS
0,1169.464647,1296.698020,"MBBS, FCPS (Orthopedic Surgery)",13.0,8,100,11,14,1,1500,Saeed surgical complex,1,1,0,0,0,0
1,1762.672335,1676.012514,"MBBS, FCPS (Nephrology)",10.0,32,94,13,7,1,1200,Advanced International Hospital,1,1,0,0,0,0
2,1151.307556,1104.541153,MBBS,12.0,0,94,14,11,0,500,No Address Available,1,0,0,0,0,0
3,1169.464647,604.753086,MBBS,2.0,12,100,14,11,1,2000,No Address Available,1,0,0,0,0,0
4,2265.768052,890.970153,"MBBS , DCH , MRCPCH (UK) , FRCPCH (UK)",20.0,122,97,15,14,1,1500,Hameedah Memorial Hospital,1,0,0,0,0,0


In [27]:
indices1, indices2 = [], []
for i in range(train.shape[0]):
    if train['Hospital Name'].iloc[i] == 'No Address Available':
        indices1.append(i)

for i in range(test.shape[0]):
    if test['Hospital Name'].iloc[i] == 'No Address Available':
        indices2.append(i)

mean_encoder_h.fit(train[['Hospital Name']], train['Fee(PKR)'])
train['Hospital Name'] = mean_encoder_h.transform(train['Hospital Name'])
test['Hospital Name'] = mean_encoder_h.transform(test['Hospital Name'])


for index in indices1:
    train['Hospital Name'].iloc[index] = 0

for index in indices2:
    test['Hospital Name'].iloc[index] = 0

C:\Users\mahmo\AppData\Local\Temp\ipykernel_13964\3899734602.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Hospital Name'].iloc[index] = 0
C:\Users\mahmo\AppData\Local\Temp\ipykernel_13964\3899734602.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Hospital Name'].iloc[index] = 0
C:\Users\mahmo\AppData\Local\Temp\ipykernel_13964\3899734602.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Hospital Nam

In [28]:
train.head()

,City,Specialization,Doctor Qualification,EXP(YRs),#Reviews,Satisfaction Rate,Avg Time to Patients(mins),Wait Time(mins),Doctors Link,Fee(PKR),Hospital Name,MBBS,FCPS,MCPS,MS,MD,FRCS
0,1169.464647,1296.698020,"MBBS, FCPS (Orthopedic Surgery)",13.0,8,100,11,14,1,1500,1238.231569,1,1,0,0,0,0
1,1762.672335,1676.012514,"MBBS, FCPS (Nephrology)",10.0,32,94,13,7,1,1200,1252.416676,1,1,0,0,0,0
2,1151.307556,1104.541153,MBBS,12.0,0,94,14,11,0,500,0.000000,1,0,0,0,0,0
3,1169.464647,604.753086,MBBS,2.0,12,100,14,11,1,2000,0.000000,1,0,0,0,0,0
4,2265.768052,890.970153,"MBBS , DCH , MRCPCH (UK) , FRCPCH (UK)",20.0,122,97,15,14,1,1500,1234.649627,1,0,0,0,0,0


In [29]:

included_features = ['City', 'EXP(YRs)', '#Reviews', 'Satisfaction Rate', 'Avg Time to Patients(mins)', 'Wait Time(mins)', 'Doctors Link', 'Hospital Name', 'Specialization']
features = train[included_features]
target = train["Fee(PKR)"]


k_best = SelectKBest(score_func=f_classif, k=6)

X = k_best.fit_transform(features, target)
y = target

#Get the indices of the selected features
selected_features_indices = k_best.get_support(indices=True)

# Get the scores associated with each feature
feature_scores = k_best.scores_

# Create a list of tuples containing feature names and scores
feature_info = list(zip(features.columns, feature_scores))

# Sort the feature info in descending order based on scores
sorted_feature_info = sorted(feature_info, key=lambda x: x[1], reverse=True)

for feature_name, feature_score in sorted_feature_info[:10]:
    print(f"{feature_name}: {feature_score:.2f}")


Specialization: 22.67
City: 17.64
#Reviews: 16.24
EXP(YRs): 15.94
Hospital Name: 12.11
Doctors Link: 9.10
Wait Time(mins): 1.79
Satisfaction Rate: 1.72
Avg Time to Patients(mins): 0.97


In [30]:
selected_features = ['City', 'Specialization', 'EXP(YRs)', 'Hospital Name', '#Reviews', 'Doctors Link']
selected_features.extend(popular_degrees)
# selected_features = df.columns
selected_features_df = train[selected_features]
test__ = test[selected_features]
selected_features_df['Fee(PKR)'] = train['Fee(PKR)']
test__['Fee(PKR)'] = test['Fee(PKR)']


C:\Users\mahmo\AppData\Local\Temp\ipykernel_13964\2858425357.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_features_df['Fee(PKR)'] = train['Fee(PKR)']
C:\Users\mahmo\AppData\Local\Temp\ipykernel_13964\2858425357.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test__['Fee(PKR)'] = test['Fee(PKR)']


In [31]:
X_train = selected_features_df.drop(['Fee(PKR)'], axis=1).values
y_train = selected_features_df['Fee(PKR)'].values
X_test = test__.drop(['Fee(PKR)'], axis=1).values
y_test = test__['Fee(PKR)'].values

RS = StandardScaler()
X_train = RS.fit_transform(X_train)
X_test = RS.transform(X_test)

In [32]:
X_train

array([[-0.31083523,  0.44410704,  0.14240817, ..., -0.255138  ,
        -0.255138  , -0.17911631],
       [ 1.38543772,  1.60099447, -0.2058298 , ..., -0.255138  ,
        -0.255138  , -0.17911631],
       [-0.36275529, -0.14196038,  0.02632885, ..., -0.255138  ,
        -0.255138  , -0.17911631],
       ...,
       [-1.3408615 ,  1.38007043,  0.72280479, ..., -0.255138  ,
        -0.255138  , -0.17911631],
       [-0.71834995, -1.66628529, -1.1344644 , ..., -0.255138  ,
        -0.255138  , -0.17911631],
       [ 0.89163118,  0.44410704,  1.76751871, ..., -0.255138  ,
        -0.255138  ,  5.5829644 ]])

In [33]:
# # Assuming X_train, y_train are your features and labels for training data
# smote = SMOTE(random_state=RANDOM_STATE)
# X_train, y_train = smote.fit_resample(X_train, y_train)

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [35]:
import time
def train_model(model):
    start_train_time = time.time()
    model.fit(X_train, y_train)
    train_preds = model.predict(X_train)
    train_accuracy = accuracy_score(y_train, train_preds)
    end_train_time = time.time()
    train_time = end_train_time - start_train_time
    print('Train Accuracy:', train_accuracy)
    print("Training time:", train_time)
def test_model(model):
    start_predict_time = time.time()
    test_preds = model.predict(X_test)
    accuracy = accuracy_score(y_test, test_preds)
    precision = precision_score(y_test, test_preds, average='weighted')
    recall = recall_score(y_test, test_preds, average='weighted')
    f1 = f1_score(y_test, test_preds, average='weighted')
    end_predict_time = time.time()
    predict_time = end_predict_time - start_predict_time
    print('Test Accuracy:', accuracy)
    print("Prediction time:", predict_time)
    print('Test Precision:', precision)
    print('Test Recall:', recall)
    print('Test F1 Score:', f1)


In [36]:
LR = LogisticRegression(C=0.5, penalty='l2', multi_class='multinomial', random_state=RANDOM_STATE)
train(LR)
test(LR)

TypeError: 'DataFrame' object is not callable

low accuracy on logistic, hence the model is not linearly seperable
in other words, linear kernel for svm will also give poor accuracy

In [ ]:
KNN = KNeighborsRegressor(n_neighbors=3)
train(KNN)
test(KNN)

ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

In [ ]:
svc = SVC(kernel='linear', random_state=RANDOM_STATE)
train(svc)
test(svc)

In [ ]:
svc_w = SVC(kernel='rbf', random_state=RANDOM_STATE, C=0.2)
train(svc_w)
test(svc_w)

In [ ]:
svc_s = SVC(kernel='sigmoid', C=0.9, random_state=RANDOM_STATE)
train(svc_s)
test(svc_s)

In [ ]:
for deg in range(6):
    svc_d = SVC(kernel='poly', degree=deg, C=0.8, random_state=RANDOM_STATE)
    train(svc_d)
    test(svc_d)

In [ ]:
RFC = RandomForestRegressor(max_depth=15, max_leaf_nodes=20,random_state=RANDOM_STATE, n_estimators=500)
train(RFC)
test(RFC)

In [ ]:

XG = GradientBoostingRegressor(max_depth=7,max_leaf_nodes=30,random_state=RANDOM_STATE, n_estimators=10)
train(XG)
test(XG)

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
EXT = ExtraTreesRegressor(random_state=RANDOM_STATE, max_depth=22)
train(EXT)
test(EXT)

In [ ]:
SGDC = SGDRegressor()
train(SGDC)
test(SGDC)

In [ ]:
ADA = AdaBoostRegressor(estimator=XG, n_estimators=150, algorithm='SAMME.R', learning_rate=0.01, )
train(ADA)
test(ADA)

In [ ]:
from sklearn.ensemble import VotingRegressor


vr = VotingRegressor(
    estimators=[('XG', XG), ('EXT', EXT), ('RFR', RFC), ('SVC', svc_w), ('KNN', KNN)],
)

train(vr)
test(vr)

In [ ]:
NB = GaussianNB(var_smoothing=0.0021)
train(NB)
test(NB)

In [ ]:
import pickle as pkl


pkl.dump(XG, open("XG_classif.pkl", "wb"))
pkl.dump(mean_encoder_c, open("mean_encoder_c_classif.pkl", "wb"))
pkl.dump(mean_encoder_s, open("mean_encoder_s_classif.pkl", "wb"))
pkl.dump(mean_encoder_h, open("mean_encoder_h_classif.pkl", "wb"))
pkl.dumb(RS, open("scaler_classif"))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

from scipy.stats import randint

model = GradientBoostingRegressor()
param_dist = {
    'n_estimators': randint(50, 200),
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
    'max_depth': randint(3, 10),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'max_features': ['auto', 'sqrt', 'log2', None]
}
GSCV = RandomizedSearchCV(model, cv=4, scoring='accuracy', param_distributions=param_dist, )

GSCV.fit(X_train, y_train)


print(GSCV.best_params_)
print(GSCV.best_score_)

In [ ]:
#'learning_rate': 0.05, 'max_depth': 6, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 6, 'n_estimators': 77, 'subsample': 1.0
model = GradientBoostingRegressor(learning_rate=0.05, max_depth=6, max_features='log2', min_samples_leaf=20, min_samples_split=10, n_estimators=80, subsample=1.0)
evaluate_model(model)